In [1]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple

In [ ]:
from dsds.metrics import roc_auc
from dsds.transform import multicat_one_hot_encode

In [ ]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.random.random(size=1_000_000) # Actuals

In [ ]:
pl.show_versions()

In [ ]:
df = pl.DataFrame({
    "text1":["abc|ggg", "abc|sss", "ccc|abc"],
    "text2":["aaa|bbb", "ccc|aaa", "bbb|ccc"]
})
print(df)

In [ ]:
print(multicat_one_hot_encode(df, cols=["text1", "text2"], delimiter="|"))

In [ ]:
test = pl.DataFrame({
    "a":[1,1,1,1]
    , "b": [None, None , "a", "a"]
})
test

In [2]:
import dsds.sample as sa
import dsds.transform as t
import polars as pl
from sklearn.datasets import make_classification

orig_x, orig_y = make_classification(n_samples = 300_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
target = "target"
features = df.columns
features.remove(target)

In [ ]:
df.groupby("target").count()

In [4]:
df = pl.DataFrame({
    "a":["a", "b", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c", "c"],
    "b":["a", "b", "c", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d"]
})
print(df)

shape: (14, 2)
┌─────┬─────┐
│ a   ┆ b   │
│ --- ┆ --- │
│ str ┆ str │
╞═════╪═════╡
│ a   ┆ a   │
│ b   ┆ b   │
│ c   ┆ c   │
│ c   ┆ d   │
│ …   ┆ …   │
│ c   ┆ d   │
│ c   ┆ d   │
│ c   ┆ d   │
│ c   ┆ d   │
└─────┴─────┘


In [6]:
print(t.merge_infreq_values(df, ["a", "b"], min_count=3))

shape: (14, 2)
┌─────┬───────┐
│ a   ┆ b     │
│ --- ┆ ---   │
│ str ┆ str   │
╞═════╪═══════╡
│ a|b ┆ a|c|b │
│ a|b ┆ a|c|b │
│ c   ┆ a|c|b │
│ c   ┆ d     │
│ …   ┆ …     │
│ c   ┆ d     │
│ c   ┆ d     │
│ c   ┆ d     │
│ c   ┆ d     │
└─────┴───────┘


In [ ]:
df.groupby(1).agg(
    pl.all().value_counts()
)

In [ ]:
for c in df.columns:
    temp = df.lazy().groupby(c).count()